**Task 07: Querying RDF(s)**

In [1]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

     |████████████████████████████████| 235kB 2.7MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 


Leemos el fichero RDF de la forma que lo hemos venido haciendo

In [2]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=N651bda138ecb49ca83b3a431e11a3ff6 (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [3]:
from rdflib.plugins.sparql import prepareQuery

case = Namespace("http://somewhere#")
print("List of subclasses with RDFLib")
for s, p, o in g.triples((None, RDFS.subClassOf, case.Person)):
  print(s)
  for sj, pd, oj in g.triples((None, RDFS.subClassOf, s)):
    print(sj)

print("List of subclasses with SPARQL")

q1 = prepareQuery('''
  SELECT 
  ?subclass ?pred ?obj 
  WHERE {
    ?subclass rdfs:subClassOf* case:Person.
    ?subclass ?pred ?obj filter(?pred not in (rdf:type))
  }
  ''', 
  initNs = { "case": case}


)

for r in g.query(q1, initBindings = { '?pred' : RDFS.subClassOf}):
  print(r.subclass, r.pred, r.obj)

List of subclasses with RDFLib
http://somewhere#Researcher
http://somewhere#PhDstudent
List of subclasses with SPARQL
http://somewhere#Researcher http://www.w3.org/2000/01/rdf-schema#subClassOf http://somewhere#Person
http://somewhere#PhDstudent http://www.w3.org/2000/01/rdf-schema#subClassOf http://somewhere#Researcher


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [15]:
from rdflib.plugins.sparql import prepareQuery

print("List of Individuals with RDFLib")
for s, p, o in g.triples((None, RDFS.subClassOf, case.Person)):
  for sj, pd, oj in g.triples((None, RDF.type, s)):
    print(sj)
    for sjt, pdt, obj in g.triples((None, RDF.type, case.Person)):
      print(sjt)
    
print("List of subclasses with SPARQL")

q2 = prepareQuery('''
  SELECT DISTINCT ?sbj 
  WHERE {
    {?person rdfs:subClassOf* case:Person.
    ?sbj a ?person.} UNION
    {?subj a case:Person.}
    
  }
  ''', 
  initNs = { "case": case}

)

for r in g.query(q2):
  print(r.sbj)

List of subclasses with RDFLib
http://somewhere#JaneSmith
http://somewhere#SaraJones
http://somewhere#JohnSmith
List of subclasses with SPARQL
http://somewhere#SaraJones
http://somewhere#JohnSmith
http://somewhere#JaneSmith


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [30]:
from rdflib.plugins.sparql import prepareQuery

print("List of Individuals and their properties with RDFLib")
for s, p, o in g.triples((None, RDFS.subClassOf, case.Person)):
  for sj, pd, oj in g.triples((None, RDF.type, s)):
    for sj_1,pd_1, ob_1 in g.triples((sj, None, None)):
      print(sj_1, pd_1, ob_1)
for sjt, pdt, obj in g.triples((None, RDF.type, case.Person)):
    for sjt_1, pdt_1, obj_1 in g.triples((sjt, None, None)):
        print(sjt, pdt_1, obj_1)

print("List of Individuals and their properties with SPARQL")

q3 = prepareQuery('''
  SELECT DISTINCT ?subj ?proper ?value
  WHERE {
    {?person rdfs:subClassOf* case:Person.
    ?subj a ?person.
    ?subj ?proper ?value.} UNION
    {?subj a case:Person.
    ?subj ?proper ?value. }
  }
  ''',
  initNs = { "case": case}


)

for r in g.query(q3):
  print(r.subj, r.proper, r.value)

List of Individuals and their properties with RDFLib
http://somewhere#JaneSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Researcher
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN jane Smith
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Jane
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John 